# [Module 4] Distributed training with horovod

Horovod는 MPI(Message Passing Interface; 메세지 전달 인터페이스)를 기반으로 하는 분산 학습 프레임워크(distributed training framework)입니다. Horovod는 TensorFlow 버전 1.12 이상에서만 사용할 수 있습니다. 자세한 내용은 [Horovod README](https://github.com/uber/horovod)에서 확인할 수 있습니다.

Horovod를 활성화하려면 학습 스크립트를 약간 수정해야 합니다. 본 실습에서 이를 직접 수행해 보겠습니다.

## Create a training script that support Horovod distributed training

`training_script/cifar10_keras_sm.py`의 사본을 생성 후 **<font color='red'>(주의: `training_script/cifar10_keras_pipe.py`의 사본이 아닙니다)</font>**, `training_script/cifar10_keras_dist.py` 로 저장하세요.

스크립트 사본을 생성하였다면 단계별로 아래의 작업들을 직접 시도합니다.

----
### TODO 1. Start horovod
Horovod에 대응하기 위해 `main()` 함수에 아래 코드를 추가합니다.

```python
    import horovod.keras as hvd
    hvd.init()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.visible_device_list = str(hvd.local_rank())
    K.set_session(tf.Session(config=config))
```

----
### TODO 2. Configure callbacks
`main()` 함수에서 callbacks을 추가합니다.

```python
    callbacks.append(hvd.callbacks.BroadcastGlobalVariablesCallback(0))
    callbacks.append(hvd.callbacks.MetricAverageCallback())
    callbacks.append(hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=5, verbose=1))
```

`hvd.rank () == 0` 에서만 실행되도록 체크포인트 및 TensorBoard 콜백을 변경해 주세요.
```python
    if hvd.rank() == 0:
        callbacks.append(ModelCheckpoint(args.output_dir + '/checkpoint-{epoch}.h5'))
```

----
### TODO 3. Configure the optimizer
Horovod에 대응하기 위해 아래의 절차들을 진행합니다.


1) `keras_model_fn` 함수에 hvd 인수를 추가합니다.
```python
# Add hvd to the function. also add it in the function call
def keras_model_fn(learning_rate, weight_decay, optimizer, momentum, hvd): 
```

2) `size=1`을 `size=hvd.size()`로 변경해 주세요.

3) 코드를 아래와 같이 수정합니다.

```python
 model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
```
바로 앞에
```python
opt = hvd.DistributedOptimizer(opt)
```
라인을 추가해 주세요.

4) `main()` 함수에서 model 인스턴스를 만들 때 hvd를 인수로 전달하도록 수정합니다.

```python
model = keras_model_fn(args.learning_rate, args.weight_decay, args.optimizer, args.momentum, hvd)
```

<font color='blue'>**본 노트북 실습에 어려움이 있다면 솔루션 파일 `training_script/cifar10_keras_dist_solution.py`을 참조하시면 됩니다.**</font>

## Run Distributed training
아래의 설정을 Estimator 객체에 전달하여 Horovod 분산 학습에 대한 설정을 할 수 있습니다.
```python
distributions = {'mpi': {
                    'enabled': True,
                    'processes_per_host': # Number of Horovod processes per host
                        }
                }
```

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

In [ ]:
prefix = 'data/DEMO-cifar10'
dataset_location = os.path.join('s3://', sagemaker_session.default_bucket(), prefix)

In [ ]:
metric_definitions = [
    {'Name': 'train:loss', 'Regex': 'loss: (.*?) '},
    {'Name': 'train:accuracy', 'Regex': 'acc: (.*?) '},
    {'Name': 'validation:loss', 'Regex': 'val_loss: (.*?) '},
    {'Name': 'validation:accuracy', 'Regex': 'val_acc: (.*?) '}
]

`train_instance_count` 인자값을 2로 설정하고 `distribution` 인자값을 추가합니다.<br>
이번에는 5 epoch 대신 10 epoch를 학습합니다.

In [ ]:
from sagemaker.tensorflow import TensorFlow

distributions = {
    'mpi': {
        'enabled': True, 
        'custom_mpi_options': '-verbose --NCCL_DEBUG=INFO',
        'processes_per_host': 1
    }
}

# Change base_job_name to 'cifar10-dist' for console visibility
estimator = TensorFlow(base_job_name='cifar10-dist',
                       entry_point='cifar10_keras_dist.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='1.14.0',
                       py_version='py3',
                       script_mode=True,                            
                       hyperparameters={'epochs': 10},
                       train_instance_count=2,   # 변경
                       train_instance_type='ml.p2.xlarge',
                       metric_definitions=metric_definitions, # 1_Monitoring_your_TensorFlow_scripts.ipynb 참조                         
                       distributions=distributions # 추가
                      )

학습 완료 후 Billable seconds도 확인해 보세요. Billable seconds는 실제로 학습 수행 시 과금되는 시간입니다.
```
Billable seconds: <time>
```

참고로, `ml.p2.xlarge` 인스턴스로 10 epoch 학습 시 전체 6분~7분이 소요되고, 실제 학습에 소요되는 시간은 3분~4분이 소요됩니다.

In [ ]:
%%time
estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)})

**잘 하셨습니다.**  

여러 분은 이제 분산 학습에 SageMaker 학습 작업을 사용할 수 있습니다.
다음 노트북으로 계속 진행하기 전에 CloudWatch 및 TensorBoard의 distribution job metrics를 살펴 보세요.
TensorBoard를 사용하여 여러분이 실행한 다른 작업을 비교할 수 있습니다.

TensorBoard 실행 시 아래의 인자값을 참조해 주세요.<br>
`--logdir dist:dist_model_dir,pipe:pipe_model_dir,file:normal_job_model_dir`